# Imports

In [11]:
import os
from functools import partial
import json
import csv

import geopandas as gpd
import matplotlib.pyplot as plt
import networkx as nx
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook
import random

from gerrychain import (
    Election,
    Graph,
    MarkovChain,
    Partition,
    accept,
    constraints,
    updaters,
)

from gerrychain.metrics import efficiency_gap, mean_median, partisan_gini
from gerrychain.proposals import recom
from gerrychain.updaters import cut_edges
from gerrychain.tree import recursive_tree_part

In [12]:
newdir = "./Outputs/"
os.makedirs(os.path.dirname(newdir + "init.txt"), exist_ok=True)
with open(newdir + "init.txt", "w") as f:
    f.write("Created Folder")

# Downloading state data

In [40]:
graph_path = "./texas.json"
graph = Graph.from_json(graph_path)

In [41]:
graph.nodes[0]

{'boundary_node': False,
 'area': 5616315.599709552,
 'CNTYVTD': '10001',
 'VTD': '0001',
 'WHITE': 2053.0,
 'OTHER': 89.0,
 'HISPANIC': 401.0,
 'TOTPOP': 3131.0,
 'VAP': 2341.0,
 'BLACK': 606.0,
 'BLKHISP': 989.0,
 'WVAP': 1677.0,
 'HISPVAP': 217.0,
 'BHVAP': 592.0,
 'BVAP': 378.0,
 'OTHVAP': 72.0,
 'COUNTY': 'Anderson',
 'FIPS': 1,
 'PRES12R': 754,
 'PRES12D': 261,
 'SEN12R': 713,
 'SEN12D': 285,
 'TOTVR12': 1629,
 'TOTTO12': 1026,
 'SEN14R': 441,
 'SEN14D': 110,
 'GOV14R': 424,
 'GOV14D': 130,
 'TOTVR14': 1712,
 'TOTTO14': 576,
 'PRES16D': 262,
 'PRES16R': 742,
 'TOTVR16': 1799,
 'TOTTO16': 1030,
 'USCD': '05',
 'SEND': 3,
 'HD': 8,
 'AREA': 6,
 'PERIM': 15}

# Chain stuff

In [42]:
election = "SEN12"
TOTPOP = "TOTPOP"

In [43]:
pop_count = 0

for i in graph.nodes:
    pop_count += graph.nodes[i][TOTPOP]
    
print(pop_count)

25145561.0


In [44]:
for n in graph.nodes:
    graph.nodes[n][election+"D"] = int(graph.nodes[n][election+"D"].replace(",",""))
    graph.nodes[n][election+"R"] = int(graph.nodes[n][election+"R"].replace(",",""))

AttributeError: 'int' object has no attribute 'replace'

In [46]:
%%time
num_dist = 36

# Exercise: Compute exact population from your data.
pop = pop_count

my_updaters = {
    "population": updaters.Tally(TOTPOP, alias="population"),
    "cut_edges": cut_edges,
    election: Election(election, {"republican": election+"R", "democratic":election+"D"})
}


new_plan = recursive_tree_part(graph,
                               range(num_dist),
                               pop/num_dist,
                               TOTPOP,
                               0.001,
                               3)
initial_partition = Partition(graph,
                              new_plan,
                              my_updaters)

CPU times: user 3min, sys: 3.88 s, total: 3min 3s
Wall time: 3min 9s


In [47]:
%%time
proposal = partial(recom,
                   pop_col = TOTPOP,
                   pop_target = pop/num_dist,
                   epsilon = 0.001,
                   node_repeats = 3)

compactness_bound = constraints.UpperBound(
    lambda p: len(p["cut_edges"]), 2 * len(initial_partition["cut_edges"])
)

CPU times: user 140 ms, sys: 13.1 ms, total: 153 ms
Wall time: 162 ms


# Pulling data from chain run

In [48]:
%%time
pop_bound = 0.01
num_steps = 10000

# last_step = Partition(graph,
#                      dict(step.assignment),
#                      my_updaters)

chain = MarkovChain(
    proposal=proposal,
    constraints=[
        constraints.within_percent_of_ideal_population(initial_partition, pop_bound),
        compactness_bound,
    ],
    accept=accept.always_accept,
    initial_state=initial_partition,
    total_steps=num_steps,
)

data = []
t = 0

with open(newdir + election + "_tx_data_specs.txt", "w") as f:
    f.write("population bound: ")
    f.write(str(pop_bound))
    f.write(", number of steps: ")
    f.write(str(num_steps))

for step in tqdm_notebook(chain):
    data.append([step[election].wins("republican"),
                         mean_median(step[election]),
                         partisan_gini(step[election]),
                         step[election].percents("republican")])
    t += 1
    if t % 1000 == 0:
        with open(newdir + election + "_tx_data" + str(t) + ".csv", "w") as f:
            writer = csv.writer(f, lineterminator="\n")
            writer.writerows(data)


CPU times: user 4h 33min 22s, sys: 43.5 s, total: 4h 34min 5s
Wall time: 5h 7min 44s
